In [56]:
import json
import re
import requests
import uuid
from bs4 import BeautifulSoup

## 37th LD position 2, 2022

In [6]:
transcript_url = 'https://www.officialhacksandwonks.com/sse-37th-ld-debate-2022#transcript'
transcript = BeautifulSoup(requests.get(transcript_url).text)

In [13]:
transcript.find_all('p')[0].text

"[00:00:09] Crystal Fincher: Welcome everyone to the South Seattle Emerald's 2022 General Election Candidate Debate. My name is Crystal Fincher. I'm a political consultant and the host of the Hacks & Wonks radio show and podcast, and I'm honored to welcome you all to tonight's debate. I'm also excited to hear from our guests running for State Representative Position 2 in the 37th Legislative District."

In [9]:
pattern = re.compile(r'\[\d+:\d+:\d+]\s(?P<name>.*):(?P<response>.*)')

In [27]:
responses = []
name = ''
for paragraph in transcript.find_all('p'):
    match = pattern.match(paragraph.text)
    if match:
        name = match.group('name')
        responses.append((name, match.group('response')))
    else:
        responses.append((name, paragraph.text))

In [36]:
cards = []
speeches = set()
for row in responses:
    speeches.add(row[0])
    cards.append(row[1])
with open('/Users/abrahamepton/code/flow/src/data/37th_debate.json', 'w+') as fh:
    fh.write(
        json.dumps(
            [{'text': response[1].strip(), 'speech': response[0].strip()} for response in responses]
        )
    )

## GOP presidential debate number 1

In [13]:
transcript_url = 'https://rollcall.com/2023/08/24/transcript-gop-presidential-hopefuls-debate-in-milwaukee/'
transcript = BeautifulSoup(requests.get(transcript_url).text)

In [31]:
pattern = re.compile(r'^(?P<name>.*): (?P<response>.*)$')

In [45]:
speaker_map = {
    'ALEXANDER DIAZ, CATHOLIC UNIVERSITY OF AMERICA': 'Bret Baier',
    'ASA HUTCHINSON (R), PRESIDENTIAL CANDIDATE': 'Asa Hutchinson',
    'BRET BAIER, FOX ANCHOR': 'Bret Baier',
    'BAIER': 'Bret Baier',
    'BURGUM': 'Doug Burgum',
    'CHRIS CHRISTIE (R), PRESIDENTIAL CANDIDATE': 'Chris Christie',
    'CHRISTIE': 'Chris Christie',
    'CANDIDATE': 'Chris Christie',
    'DESANTIS': 'Ron DeSantis',
    'DOUG BURGUM (R), 2024 PRESDIENTIAL CANDIDATE': 'Doug Burgum',
    'GOV. RON DESANTIS (R-FL), PRESIDENTIAL CANDIDAITE': 'Ron DeSantis',
    'HALEY': 'Nikki Haley',
    'HUTCHINSON': 'Asa Hutchinson',
    'JOE BIDEN, PRESIDENT OF THE UNITED STATES': 'Martha MacCallum',
    'BIDEN': 'Martha MacCallum',
    'MACCALLUM': 'Martha MacCallum',
    'MARTHA MACCALLUM, FOX ANCHOR': 'Martha MacCallum',
    'MIKE PENCE (R), 2024 PRESIDENTIAL CANDIDATE': 'Mike Pence',
    'NIKKI HALEY (R), 2024 PRESDIENTIAL CANDIDATE': 'Nikki Haley',
    'PENCE': 'Mike Pence',
    'RAMASWAMY': 'Vivek Ramaswamy',
    'SCOTT': 'Tim Scott',
    'SEN. TIM SCOTT (R-SC), PRESIDENTIAL CANDIDATE': 'Tim Scott',
    'UNKNOWN': 'Bret Baier',
    'VIVEK RAMASWAMY (R), 2024 PRESDIENTIAL CANDIDATE': 'Vivek Ramaswamy'
}
moderators = ['Bret Baier', 'Martha MacCallum']

In [49]:
responses = []
name = ''
for paragraph in transcript.find_all('p'):
    if paragraph.text.startswith('(') or paragraph.text == 'CHRIS CHRISTIE (R), FORMER NEW JERSEY GOVERNOR, PRESIDENTIAL':
        continue
    match = pattern.match(paragraph.text)
    if match:
        name = match.group('name')
        if name and name not in speaker_map:
            continue
        responses.append((speaker_map[name], match.group('response')))
    else:
        if name in speaker_map:
            responses.append((speaker_map[name], paragraph.text))

In [51]:
with open('src/data/37th_debate_height.json') as fh:
    data = json.loads(fh.read())

In [85]:
speech_map = {}
speeches = []
for r in responses:
    if r[0] not in speech_map:
        speech_map[r[0]] = str(uuid.uuid4())
        speeches.append({'label': r[0], 'id': speech_map[r[0]]})
gop_data = {
    'cards': [{'speech': speech_map[r[0]], 'text': r[1], 'id': str(uuid.uuid4())} for r in responses],
    'edges': [],
    'title': 'GOP Presidential Debate #1',
    'meta': '',
    'tags': {},
    'speeches': speeches,
    'date': 'Aug. 23, 2023',
    'source': 'CQ via Roll Call',
    'url': 'https://rollcall.com/2023/08/24/transcript-gop-presidential-hopefuls-debate-in-milwaukee/',
    'moderators': [speech_map[m] for m in moderators]
}
for tag in data['tags']:
    gop_data['tags'][tag] = []

In [86]:
for idx, card in enumerate(gop_data['cards']):
    if idx < len(gop_data['cards']) - 1:
        gop_data['edges'].append(
            {
                'source': f'card_{card["id"]}',
                'target': f'card_{gop_data["cards"][idx + 1]["id"]}'
            }
        )

In [87]:
with open('src/data/gop_debate_1.json', 'w+') as fh:
    fh.write(json.dumps(gop_data))